In [249]:
import numpy as np
import networkx as nx
from pyvis.network import Network
import math
import community as community_louvain

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import xmltodict
import os
import requests
from bs4 import BeautifulSoup
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [250]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})

<Response [200]>

In [251]:
env_name = 'Nitrate_as_reduction_target'

In [252]:
condition_df = pd.read_excel("ecoli/experiments/new_environments_test/done.xlsx")
condition_df["Add"] = condition_df["Add"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Remove"] = condition_df["Remove"].apply(lambda x: set(str(x).split(", ") if x == x else set()))

row_num = condition_df[condition_df['Environment'] == env_name].index[0]
important_add = condition_df["Add"].loc[row_num]
important_remove = condition_df["Remove"].loc[row_num]

In [253]:
time = '50'
date = '2023-07-21'
experiment = 'fba_new_environments'
entry = f'{experiment}_{env_name}_{time}_{date}'
folder = f'out/fba_new_env/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry

output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

metabolites_idx = {species: i for i, species in enumerate(metabolism.metabolite_names)}
index_to_met = {v: k for k, v in metabolites_idx.items()}

reaction_idx = {reaction: i for i, reaction in enumerate(metabolism.reaction_names)}
index_to_rxn = {v: k for k, v in reaction_idx.items()}

metabolism.catalyst_ids = metabolism.parameters['catalyst_ids']
catalyst_idx = {catalyst: i for i, catalyst in enumerate(metabolism.catalyst_ids)}

In [254]:
heatmap_df = pd.read_excel("out/new_env_analysis/pathway_heatmap.xlsx")
heatmap_df = heatmap_df[heatmap_df[env_name] != 0]

In [255]:
G = nx.DiGraph()

In [277]:
def find_connecting_reactions(one, two):
    one_reactions = []
    two_reactions = []
    try:
        req_str = f"https://websvc.biocyc.org/getxml?id=ECOLI:{one}&detail=low"
        r = s.get(req_str)
        o = xmltodict.parse(r.content)['ptools-xml']['Pathway']['reaction-list']['Reaction']
        if type(o) is dict:
            o = [o]

        for rxn in o:
            one_reactions.append(rxn['@frameid'])

        req_str = f"https://websvc.biocyc.org/getxml?id=ECOLI:{two}&detail=low"
        r = s.get(req_str)
        o = xmltodict.parse(r.content)['ptools-xml']['Pathway']['reaction-list']['Reaction']
        if type(o) is dict:
            o = [o]

        for rxn in o:
            two_reactions.append(rxn['@frameid'])
    except Exception:
        pass

    intersection = set(one_reactions).intersection(set(two_reactions))
    print(intersection)

In [279]:
find_connecting_reactions("PWY0-1335", "PWY-7219")

set()


In [278]:
def get_pathway_name(pathway):
    try:
        req_str = f"https://websvc.biocyc.org/getxml?id=ECOLI:{pathway}&detail=low"
        r = s.get(req_str)
        o = xmltodict.parse(r.content)['ptools-xml']['Pathway']['common-name']['#text']
        if type(o) is dict:
            o = [o]

        return o
    except KeyError:
        pass
    except Exception:
        pass

In [257]:
# Change K here to select the top k reactions/pathways to plot
k = 100
heatmap_df.sort_values(by=[env_name], ascending=False, inplace=True)
top_increase = [heatmap_df.iloc[i, 1] for i in range(k)]
bottom_increase = [heatmap_df.iloc[i, 1] for i in range(-1, -1*k - 1, -1)]

if "Non-zero Values for each environment" in top_increase:
    top_increase.remove("Non-zero Values for each environment")
elif "Non-zero Values for each environment" in bottom_increase:
    bottom_increase.remove("Non-zero Values for each environment")

count = 0
for pathway in top_increase:
    row = heatmap_df[heatmap_df['Pathways'] == pathway].index[0]
    flux = heatmap_df[env_name].loc[row]
    name = get_pathway_name(pathway)

    color = "green"
    if flux < 0 or abs(flux) < 0.1:
        break

    G.add_node(count, label=name, size=abs(flux*30), color=color, title=pathway)
    count += 1

for pathway in bottom_increase:
    row = heatmap_df[heatmap_df['Pathways'] == pathway].index[0]
    flux = heatmap_df[env_name].loc[row]
    name = get_pathway_name(pathway)

    color = "red"
    if flux > 0 or abs(flux) < 0.1:
        break

    G.add_node(count, label=pathway, size=abs(flux*30), color=color, title=pathway)
    count += 1


In [258]:
all_nodes = list(G.nodes(data=True))
for node in all_nodes:
    print(node[1])

{'label': 'NADH to fumarate electron transfer', 'size': 71.2772610480168, 'color': 'green', 'title': 'PWY0-1336'}
{'label': 'NADH to dimethyl sulfoxide electron transfer', 'size': 71.2772610480168, 'color': 'green', 'title': 'PWY0-1348'}
{'label': 'nitrate reduction III (dissimilatory)', 'size': 61.8855509774106, 'color': 'green', 'title': 'PWY0-1321'}
{'label': 'L-alanine degradation I', 'size': 56.269925742461396, 'color': 'green', 'title': 'ALADEG-PWY'}
{'label': 'L-aspartate degradation III (anaerobic)', 'size': 38.9882754774837, 'color': 'green', 'title': 'PWY-8294'}
{'label': 'UMP biosynthesis I', 'size': 12.63840817311879, 'color': 'green', 'title': 'PWY-5686'}
{'label': 'glycolysis II (from fructose 6-phosphate)', 'size': 8.54316559689027, 'color': 'green', 'title': 'PWY-5484'}
{'label': 'superpathway of glycolysis, pyruvate dehydrogenase, TCA, and glyoxylate bypass', 'size': 8.15241469630458, 'color': 'green', 'title': 'GLYCOLYSIS-TCA-GLYOX-BYPASS'}
{'label': 'glycolysis I (fr

In [259]:
nt = Network('1000px', '1500px', notebook=True, cdn_resources='in_line', directed=True)
nt.from_nx(G)

nt.show_buttons(filter_=['physics'])
nt.show(f'out/new_env_analysis/pathway_only/{env_name}_pathway_{k}_graph.html')

out/new_env_analysis/pathway_only/Nitrate_as_reduction_target_pathway_100_graph.html
